In [78]:
from bs4 import BeautifulSoup
import requests
import csv
import re
import datetime
from dateutil.parser import parse
from dateutil.tz import gettz

tzinfos = {"ET": gettz("America/New_York")}

In [117]:
def parseBoxScore(link):
    gamepage = requests.get('https://www.baseball-reference.com' + link['href']).text
    gamesoup = BeautifulSoup(gamepage,features='html.parser')
    scorebox = gamesoup.find('div', { 'class': 'scorebox_meta' })
    divs = scorebox.find_all('div')
    time, venue, duration, date = None, None, None, None
    for div in divs:
        if (div.text.startswith('Start Time: ')):
            time = div.text[12:]
        elif (div.text.startswith('Venue: ')):
            venue = div.text[7:]
        elif (div.text.startswith('Game Duration: ')):
            duration = div.text[15:]
        elif (div.text.startswith('Monday')):
            date = div.text[8:]
        elif (div.text.startswith('Tuesday')):
            date = div.text[9:]
        elif (div.text.startswith('Wednesday')):
            date = div.text[11:]
        elif (div.text.startswith('Thursday')):
            date = div.text[10:]
        elif (div.text.startswith('Friday')):
            date = div.text[8:]
        elif (div.text.startswith('Saturday')):
            date = div.text[10:]
        elif (div.text.startswith('Sunday')):
            date = div.text[8:]
    if (time == None or date == None or venue == None or duration == None):
        print(link)
    parsedDate = parse(date + ' ' + time, tzinfos=tzinfos)
    colonIndex = duration.index(':')
    hours = duration[0:colonIndex]
    minutes = duration[colonIndex + 1:]
    endDate = parsedDate + datetime.timedelta(minutes=int(minutes),hours=int(hours))
    return [parsedDate, endDate, venue]

In [118]:
page = requests.get('https://www.baseball-reference.com/leagues/MLB/2018-schedule.shtml').text
soup = BeautifulSoup(page,features='html.parser')
links = soup.find_all('a', href=True, text='Boxscore')
scores = []
for link in links:
    scores.append(parseBoxScore(link))
nparray = np.array(scores)
np.savetxt('scores-times-and-venues.csv',nparray,delimiter=',',fmt='%s')


<a href="/boxes/TOR/TOR201804172.shtml">Boxscore</a>


TypeError: can only concatenate str (not "NoneType") to str